In [16]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [17]:
df_location = pd.read_excel('../../python/df_location.xlsx')
df_distance = pd.read_excel('../../python/df_distance_km.xlsx')
df_vehicle = pd.read_excel('../../python/df_vehicle.xlsx')
df_orders = pd.read_excel('../../python/df_orders.xlsx')

In [18]:
clients_names = df_location['cliente'].to_list()
clients_coords = df_location[['Latitud','Longitud']].to_numpy()

clients = {name: coord for name, coord in zip(clients_names, clients_coords)}

In [19]:
distances_matrix = df_distance.to_numpy()

In [20]:
def calculate_route_distance(route, distance_matrix, cost_per_unit):
    distance = 0
    for i in range(len(route) - 1):
        dist = distance_matrix[route[i]][route[i + 1]]
        if dist == 0:
            return float('inf')
        distance += dist * cost_per_unit
    return distance

In [21]:
def is_valid_route(route, demandas, capacity):
    total_demand = sum(demandas[cliente] for cliente in route)
    return total_demand <= capacity

In [22]:
def generate_valid_initial_routes(distance_matrix, num_vehiculos, almacen, demandas, capacidades):
    clientes = [cliente for cliente in range(len(distance_matrix)) if cliente != almacen]
    valid_routes_found = False
    while not valid_routes_found:
        random.shuffle(clientes)
        routes = [[almacen] for _ in range(num_vehiculos)]
        for cliente in clientes:
            for route in routes:
                if sum(demandas[r] for r in route) + demandas[cliente] <= capacidades[routes.index(route)]:
                    route.append(cliente)
                    break
        for route in routes:
            route.append(almacen)
        if all(calculate_route_distance(route, distance_matrix, 1) != float('inf') for route in routes):
            valid_routes_found = True
    return routes


In [23]:
def calculate_total_distance(routes, distance_matrix, costes):
    total_distance = 0
    for route, cost in zip(routes, costes):
        total_distance += calculate_route_distance(route, distance_matrix, cost)
    return total_distance

In [24]:
def generate_vecinos_multiple(routes, almacen, demandas, capacidades):
    neighbors = []
    for i in range(len(routes)):
        for j in range(len(routes)):
            if i != j:
                for k in range(1, len(routes[i]) - 1):
                    for l in range(1, len(routes[j]) - 1):
                        neighbor = [route[:] for route in routes]
                        neighbor[i][k], neighbor[j][l] = neighbor[j][l], neighbor[i][k]
                        if is_valid_route(neighbor[i], demandas, capacidades[i]) and is_valid_route(neighbor[j], demandas, capacidades[j]):
                            neighbors.append(neighbor)
    return neighbors

In [25]:
def tabu_search_multiple_tsp(distance_matrix, initial_routes, max_iterations, tabu_size, almacen, costes, demandas, capacidades, max_no_improve=20):
    current_routes = initial_routes
    best_routes = current_routes
    best_distance = calculate_total_distance(current_routes, distance_matrix, costes)
    tabu_list = []
    historial = []
    no_improve_counter = 0

    for iteration in range(max_iterations):
        neighbors = generate_vecinos_multiple(current_routes, almacen, demandas, capacidades)
        evaluated_neighbors = [(neighbor, calculate_total_distance(neighbor, distance_matrix, costes)) 
                               for neighbor in neighbors if neighbor not in tabu_list]

        if not evaluated_neighbors:
            break  # Si no hay vecinos válidos, detener el algoritmo

        best_neighbor, best_neighbor_distance = min(evaluated_neighbors, key=lambda x: x[1])
        
        current_routes = best_neighbor
        if best_neighbor_distance < best_distance:
            best_routes = best_neighbor
            best_distance = best_neighbor_distance
            no_improve_counter = 0
        else:
            no_improve_counter += 1 

        # Actualizar lista Tabú
        tabu_list.append(best_neighbor)
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Guardar historial
        historial.append((best_neighbor, best_neighbor_distance))

        # Verificar criterio de parada adicional
        if no_improve_counter >= max_no_improve:
            print(f"Parada anticipada: {no_improve_counter} iteraciones sin mejora.")
            break

    return best_routes, best_distance, historial



In [26]:
def convert_routes_to_city_names(routes, city_names):
    return [[city_names[city] for city in route] for route in routes]

In [27]:
almacen = len(clients_names) - 1
num_vehicles = 5
costes = df_vehicle['costo_km'].to_list()
capacidades = df_vehicle['capacidad_kg'].to_list()
demandas_dict = dict(zip(df_orders["cliente"], df_orders["order_demand"]))
demandas = [demandas_dict.get(client, 0) for client in clients_names]

initial_routes = generate_valid_initial_routes(distances_matrix, num_vehicles, almacen, demandas, capacidades)


In [28]:
# Parámetros iniciales
max_iterations = 100
tabu_size = 10
max_no_improve = 20  # Número máximo de iteraciones sin mejora

# Ejecutar Tabu Search
mejor_solucion, costo_mejor_solucion, historial = tabu_search_multiple_tsp(
    distances_matrix, initial_routes, max_iterations, tabu_size, almacen, costes, demandas, capacidades, max_no_improve=max_no_improve
)


Parada anticipada: 20 iteraciones sin mejora.


In [29]:
# Guardar el modelo
modelo_tabu = {
    "mejor_solucion": mejor_solucion,
    "costo_mejor_solucion": costo_mejor_solucion,
    "parametros": {
        "tamano_tabu": tabu_size,
        "max_iteraciones": max_iterations,
        "criterio_parada": f"Sin mejora en {max_no_improve} iteraciones"
    },
    "historial": historial
}

In [30]:
print(modelo_tabu)

{'mejor_solucion': [[20, 10, 13, 20], [20, 15, 2, 18, 19, 20], [20, 0, 9, 11, 16, 4, 20], [20, 17, 14, 7, 20], [20, 8, 3, 6, 1, 12, 5, 20]], 'costo_mejor_solucion': np.float64(32.440489), 'parametros': {'tamano_tabu': 10, 'max_iteraciones': 100, 'criterio_parada': 'Sin mejora en 20 iteraciones'}, 'historial': [([[20, 10, 0, 20], [20, 3, 17, 18, 14, 20], [20, 7, 9, 11, 16, 4, 20], [20, 19, 2, 13, 20], [20, 8, 15, 6, 1, 12, 5, 20]], np.float64(39.383562)), ([[20, 10, 0, 20], [20, 7, 17, 18, 14, 20], [20, 3, 9, 11, 16, 4, 20], [20, 19, 2, 13, 20], [20, 8, 15, 6, 1, 12, 5, 20]], np.float64(36.36910399999999)), ([[20, 10, 8, 20], [20, 7, 17, 18, 14, 20], [20, 3, 9, 11, 16, 4, 20], [20, 19, 2, 13, 20], [20, 0, 15, 6, 1, 12, 5, 20]], np.float64(35.902924)), ([[20, 10, 13, 20], [20, 7, 17, 18, 14, 20], [20, 3, 9, 11, 16, 4, 20], [20, 19, 2, 8, 20], [20, 0, 15, 6, 1, 12, 5, 20]], np.float64(35.203245)), ([[20, 10, 13, 20], [20, 7, 17, 18, 14, 20], [20, 0, 9, 11, 16, 4, 20], [20, 19, 2, 8, 20], 